# Setting Up a BigQuery Sandbox Environment

## Introduction
This notebook guides you through the process of setting up a BigQuery Sandbox environment using a free account. The BigQuery Sandbox allows users to explore the capabilities of Google's BigQuery service without the need for a billing account.

## Steps to Set Up BigQuery Sandbox
1. Visit the [Google Cloud Console](https://console.cloud.google.com/).
2. Sign in with your Google account. If you do not have a Google account, you will need to create one.
3. Navigate to the BigQuery Console.
4. The BigQuery Sandbox environment is automatically activated when you visit the BigQuery Console for the first time with your Google account.
5. Start using BigQuery without providing a credit card or enabling billing for your project.

## Limitations of BigQuery Sandbox
BigQuery Sandbox comes with certain limitations in terms of storage and compute resources. Here are the key limitations:
- **Storage:** You can store up to 10 GB of data at no cost.
- **Query Processing:** You can process up to 1 TB of data per month at no cost.

Note: Once you reach these limits, you will need to upgrade to a paid account to continue using BigQuery services.

### Simple SELECT with aliases


In [ ]:
%%bigquery
SELECT * FROM (SELECT "apple" AS fruit, "carrot" AS vegetable);


### UNION ALL basics


In [ ]:
%%bigquery
SELECT "apple" AS fruit
UNION ALL SELECT "banana"
UNION ALL SELECT "cherry";


### WHERE filter on inline data


In [ ]:
%%bigquery
SELECT * FROM (
  SELECT "apple" AS fruit, 10 AS qty UNION ALL
  SELECT "banana", 5 UNION ALL
  SELECT "cherry", 12
)
WHERE qty > 8;


### ORDER BY descending


In [ ]:
%%bigquery
SELECT * FROM (
  SELECT "apple" AS fruit, 10 AS qty UNION ALL
  SELECT "banana", 5 UNION ALL
  SELECT "cherry", 12
)
ORDER BY qty DESC;


### GROUP BY and COUNT


In [ ]:
%%bigquery
SELECT fruit, COUNT(*) AS cnt
FROM (
  SELECT "apple" AS fruit UNION ALL
  SELECT "banana" UNION ALL
  SELECT "apple"
)
GROUP BY fruit;


### CASE WHEN for labeling


In [ ]:
%%bigquery
SELECT fruit,
  CASE WHEN qty >= 10 THEN "High" ELSE "Low" END AS stock_level
FROM (
  SELECT "apple" AS fruit, 10 AS qty UNION ALL
  SELECT "banana", 5
);


### DISTINCT values


In [ ]:
%%bigquery
SELECT DISTINCT fruit
FROM (
  SELECT "apple" AS fruit UNION ALL
  SELECT "banana" UNION ALL
  SELECT "apple"
);


### LIMIT and OFFSET


In [ ]:
%%bigquery
SELECT fruit
FROM UNNEST(["apple","banana","cherry","date","mango"]) AS fruit
ORDER BY fruit
LIMIT 3 OFFSET 1;


### IN and BETWEEN


In [ ]:
%%bigquery
SELECT * FROM (
  SELECT "apple" AS fruit, 10 AS qty UNION ALL
  SELECT "banana", 5 UNION ALL
  SELECT "cherry", 12
)
WHERE fruit IN ("apple","cherry") AND qty BETWEEN 10 AND 20;


### LIKE and REGEXP_CONTAINS


In [ ]:
%%bigquery
SELECT fruit
FROM UNNEST(["apple","banana","cherry","apricot"]) AS fruit
WHERE fruit LIKE "ap%"
   OR REGEXP_CONTAINS(fruit, r"an");


### Math and string functions


In [ ]:
%%bigquery
SELECT
  "apple" AS fruit,
  LENGTH("apple") AS name_len,
  10 + 5 AS total,
  CONCAT("fresh_", "fruit") AS tag;


### Array to rows with UNNEST


In [ ]:
%%bigquery
SELECT fruit
FROM UNNEST(["apple","banana","cherry"]) AS fruit;


### Structs (rows) with UNNEST


In [ ]:
%%bigquery
SELECT item.name, item.price
FROM UNNEST([STRUCT("apple" AS name, 1.2 AS price),
             STRUCT("banana", 0.8),
             STRUCT("cherry", 2.1)]) AS item;


### Arrays of STRUCT and filtering


In [ ]:
%%bigquery
SELECT i.name, i.price
FROM UNNEST([STRUCT("apple" AS name, 1.2 AS price),
             STRUCT("banana", 0.8),
             STRUCT("cherry", 2.1)]) AS i
WHERE i.price > 1;


### WITH (CTE) basics


In [ ]:
%%bigquery
WITH fruits AS (
  SELECT "apple" AS fruit, 10 AS qty UNION ALL
  SELECT "banana", 5 UNION ALL
  SELECT "cherry", 12
)
SELECT fruit, qty*2 AS double_qty
FROM fruits;


### Aggregations with HAVING


In [ ]:
%%bigquery
WITH sales AS (
  SELECT "apple" AS fruit, 2 AS units UNION ALL
  SELECT "apple", 3 UNION ALL
  SELECT "banana", 4 UNION ALL
  SELECT "banana", 1
)
SELECT fruit, SUM(units) AS total_units
FROM sales
GROUP BY fruit
HAVING SUM(units) >= 5;


### JOIN two inline tables


In [ ]:
%%bigquery
WITH prices AS (
  SELECT "apple" AS fruit, 1.2 AS price UNION ALL
  SELECT "banana", 0.8
),
stock AS (
  SELECT "apple" AS fruit, 10 AS qty UNION ALL
  SELECT "banana", 5
)
SELECT p.fruit, p.price, s.qty, p.price * s.qty AS inventory_value
FROM prices p
JOIN stock s
USING (fruit);


### LEFT JOIN with missing match


In [ ]:
%%bigquery
WITH A AS (
  SELECT 1 AS id, "Alice" AS name UNION ALL
  SELECT 2, "Bob"
),
B AS (
  SELECT 2 AS id, "Bangalore" AS city
)
SELECT A.id, A.name, B.city
FROM A LEFT JOIN B USING (id);


### Cross join with UNNEST to split strings


In [ ]:
%%bigquery
WITH data AS (
  SELECT "apple,banana,cherry" AS csv
)
SELECT part
FROM data, UNNEST(SPLIT(csv, ",")) AS part;


### Window function: ROW_NUMBER


In [ ]:
%%bigquery
WITH s AS (
  SELECT "apple" AS fruit, 10 AS qty UNION ALL
  SELECT "banana", 5 UNION ALL
  SELECT "cherry", 12 UNION ALL
  SELECT "banana", 9
)
SELECT fruit, qty,
       ROW_NUMBER() OVER (PARTITION BY fruit ORDER BY qty DESC) AS rn
FROM s;


### QUALIFY to keep top-1 per group


In [ ]:
%%bigquery
WITH s AS (
  SELECT "apple" AS fruit, 10 AS qty UNION ALL
  SELECT "banana", 5 UNION ALL
  SELECT "cherry", 12 UNION ALL
  SELECT "banana", 9
)
SELECT fruit, qty
FROM s
QUALIFY ROW_NUMBER() OVER (PARTITION BY fruit ORDER BY qty DESC) = 1;


### COALESCE and IFNULL


In [ ]:
%%bigquery
SELECT
  COALESCE(NULL, "fallback") AS coalesced,
  IFNULL(NULL, "default") AS ifnull_val;


### SAFE_CAST (no errors)


In [ ]:
%%bigquery
SELECT
  SAFE_CAST("123" AS INT64) AS ok_int,
  SAFE_CAST("abc" AS INT64) AS null_int;


### DATE and TIMESTAMP creation


In [ ]:
%%bigquery
SELECT
  DATE(2025, 8, 21) AS d,
  TIMESTAMP("2025-08-21 10:00:00+00") AS ts;


### DATE functions


In [ ]:
%%bigquery
SELECT
  DATE "2025-08-21" AS d,
  EXTRACT(YEAR FROM DATE "2025-08-21") AS yr,
  DATE_ADD(DATE "2025-08-21", INTERVAL 7 DAY) AS plus_7d;


### String to date parse with PARSE_DATE


In [ ]:
%%bigquery
SELECT
  PARSE_DATE("%Y/%m/%d", "2025/08/21") AS d;


### ARRAY functions: length and contains


In [ ]:
%%bigquery
SELECT
  ARRAY_LENGTH([1,2,3]) AS len,
  2 IN UNNEST([1,2,3]) AS has_two;


### Build array from rows with ARRAY_AGG


In [ ]:
%%bigquery
WITH items AS (
  SELECT "apple" AS fruit, 10 AS qty UNION ALL
  SELECT "banana", 5 UNION ALL
  SELECT "cherry", 12
)
SELECT ARRAY_AGG(fruit ORDER BY qty DESC) AS fruits_by_qty
FROM items;


### Subquery in SELECT


In [ ]:
%%bigquery
SELECT
  (SELECT COUNT(*) FROM UNNEST([1,2,3,4])) AS cnt,
  (SELECT MAX(x) FROM UNNEST([1,9,2]) AS x) AS mx;


### Scalar subquery in WHERE


In [ ]:
%%bigquery
SELECT fruit, qty
FROM (
  SELECT "apple" AS fruit, 10 AS qty UNION ALL
  SELECT "banana", 5 UNION ALL
  SELECT "cherry", 12
)
WHERE qty > (SELECT AVG(q) FROM (SELECT 10 AS q UNION ALL SELECT 5 UNION ALL SELECT 12));


### SELECT * REPLACE to modify a column


In [ ]:
%%bigquery
WITH items AS (
  SELECT "apple" AS fruit, 10 AS qty
)
SELECT * REPLACE ( (qty*2) AS qty )
FROM items;


### Named STRUCT fields and dot access


In [ ]:
%%bigquery
SELECT
  s.name, s.details.size
FROM UNNEST([STRUCT("tshirt" AS name, STRUCT("M" AS size, "blue" AS color) AS details)]) AS s;


### Pivot-like aggregation with CASE


In [ ]:
%%bigquery
WITH sales AS (
  SELECT "apple" AS fruit, "store1" AS store, 5 AS units UNION ALL
  SELECT "apple","store2", 7 UNION ALL
  SELECT "banana","store1", 3
)
SELECT
  fruit,
  SUM(CASE WHEN store="store1" THEN units ELSE 0 END) AS store1_units,
  SUM(CASE WHEN store="store2" THEN units ELSE 0 END) AS store2_units
FROM sales
GROUP BY fruit;


### String functions: SPLIT / CONCAT / UPPER


In [ ]:
%%bigquery
SELECT
  SPLIT("a,b,c", "," ) AS parts,
  CONCAT("Big", "Query") AS joined,
  UPPER("hello") AS uppered;


### Numeric rounding functions


In [ ]:
%%bigquery
SELECT
  ROUND(3.14159, 2) AS r2,
  CEIL(2.01) AS c,
  FLOOR(2.99) AS f;


### Boolean logic with AND/OR/NOT


In [ ]:
%%bigquery
SELECT
  TRUE AND FALSE AS t_and_f,
  TRUE OR FALSE AS t_or_f,
  NOT TRUE AS not_t;


### Window function: moving average


In [ ]:
%%bigquery
WITH s AS (
  SELECT 1 AS day, 10 AS val UNION ALL
  SELECT 2, 20 UNION ALL
  SELECT 3, 30 UNION ALL
  SELECT 4, 40
)
SELECT
  day, val,
  AVG(val) OVER (ORDER BY day ROWS BETWEEN 1 PRECEDING AND CURRENT ROW) AS mov_avg
FROM s;


### QUALIFY with RANK for top-N


In [ ]:
%%bigquery
WITH s AS (
  SELECT "A" AS grp, 10 AS score UNION ALL
  SELECT "A", 5 UNION ALL
  SELECT "A", 8 UNION ALL
  SELECT "B", 12 UNION ALL
  SELECT "B", 9
)
SELECT grp, score
FROM s
QUALIFY RANK() OVER (PARTITION BY grp ORDER BY score DESC) <= 2;


### Safe division to avoid division-by-zero


In [ ]:
%%bigquery
WITH s AS (
  SELECT "apple" AS fruit, 10 AS qty, 2 AS boxes UNION ALL
  SELECT "banana", 5, 0
)
SELECT fruit, qty, boxes,
  IF(boxes = 0, NULL, qty / boxes) AS per_box
FROM s;


### IF / ELSEIF via CASE


In [ ]:
%%bigquery
SELECT n,
  CASE
    WHEN n % 15 = 0 THEN "FizzBuzz"
    WHEN n % 3 = 0 THEN "Fizz"
    WHEN n % 5 = 0 THEN "Buzz"
    ELSE CAST(n AS STRING)
  END AS label
FROM UNNEST([1,2,3,4,5,15]) AS n;


### Datetime extraction


In [ ]:
%%bigquery
SELECT
  DATETIME "2025-08-21 14:35:00" AS dt,
  EXTRACT(HOUR FROM DATETIME "2025-08-21 14:35:00") AS hr;


### Working with NULLs


In [ ]:
%%bigquery
SELECT
  NULL AS a,
  IFNULL(NULL, "fallback") AS b,
  NULLIF("x","x") AS c;


### ARRAY subscript and OFFSET/ORDINAL


In [ ]:
%%bigquery
WITH arr AS (
  SELECT [10,20,30] AS a
)
SELECT
  a[OFFSET(0)] AS first0,
  a[ORDINAL(2)] AS second1_based
FROM arr;


### Generate rows with UNNEST and filter


In [ ]:
%%bigquery
SELECT n
FROM UNNEST([1,2,3,4,5,6,7,8,9,10]) AS n
WHERE n % 2 = 0;


### LEAST/GREATEST functions


In [ ]:
%%bigquery
SELECT
  LEAST(10, 5, 7) AS mn,
  GREATEST(10, 5, 7) AS mx;
